In [2]:
import requests
print(requests)

<module 'requests' from '/usr/local/lib/python3.11/dist-packages/requests/__init__.py'>


In [ ]:
ApiNewsKey=

In [59]:
async def get_news():
    keyword = input("검색어를 입력하세요 : ")
    url = f"https://newsapi.org/v2/everything?q={keyword}&language=ko&apiKey={ApiNewsKey}"
    response = requests.get(url).json()
    for i in response['articles']:
        print(i['title'],i['description'])


In [49]:
!pip install python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 10.1 MB/s eta 0:00:00


In [ ]:

import logging
import httpx  # API 호출을 위한 라이브러리 임포트
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import nest_asyncio
import asyncio

# 1. 봇 토큰 설정 (아까 BotFather에게서 받은 토큰을 여기에 붙여넣으세요!)
TOKEN = 

# 로깅 설정
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logging.getLogger("httpx").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)


# --- 새로운 부분: API 호출 함수 정의 ---
async def get_api_data() -> str:
    """
    외부 API를 호출하여 데이터를 가져오는 비동기 함수.
    이 함수는 예시이며, 실제 사용할 API에 맞게 수정해야 합니다.
    """
    # **중요:** 여기에 실제 사용할 API의 URL과 필요한 API 키를 넣으세요.
    # 이 예시는 OpenWeatherMap API의 대략적인 구조를 따릅니다.
    # 실제 API 키는 반드시 발급받아야 합니다!
    api_url = "https://api.openweathermap.org/data/2.5/weather"
    city_name = "Wonju"  # 원하는 도시 이름
    # **YOUR_OPENWEATHERMAP_API_KEY_HERE** 부분을 당신의 실제 API 키로 교체하세요!
    api_key = #openweather api키

    params = {
        "q": city_name,
        "appid": api_key,
        "units": "metric",  # 섭씨 온도를 원하면 'metric', 화씨는 'imperial'
        "lang": "kr"  # 한국어 설명을 원하면 'kr'
    }

    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, params=params, timeout=10)  # 10초 타임아웃
            response.raise_for_status()  # HTTP 오류 (4xx, 5xx)가 발생하면 예외 발생

            data = response.json()
            # API 응답에서 원하는 정보를 추출하여 문자열로 포맷팅합니다.
            # 이 부분은 사용하는 API의 응답 구조에 따라 달라집니다.

            # OpenWeatherMap 예시 응답 구조를 기반으로 데이터 추출
            weather_description = data['weather'][0]['description']
            temperature = data['main']['temp']
            feels_like = data['main']['feels_like']
            humidity = data['main']['humidity']

            return (f"현재 {city_name} 날씨 정보:\n"
                    f"✨ 날씨: {weather_description}\n"
                    f"🌡️ 온도: {temperature}°C (체감: {feels_like}°C)\n"
                    f"💧 습도: {humidity}%\n")

        except httpx.RequestError as e:
            logger.error(f"API 요청 중 오류 발생: {e}")
            return "날씨 정보를 가져오는 데 실패했습니다. 네트워크 연결을 확인하거나 API 키를 점검해주세요. 😞"
        except KeyError as e:
            logger.error(f"API 응답에서 키 오류 발생: {e}. 응답 구조가 예상과 다릅니다.")
            return "날씨 정보를 처리하는 데 문제가 발생했습니다. (데이터 형식 오류) 🧐"
        except Exception as e:
            logger.error(f"예상치 못한 오류 발생: {e}")
            return "날씨 정보를 가져오는 중 알 수 없는 오류가 발생했습니다. 🥺"

async def news_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    ApiNewsKey=#뉴스apikey
    if not context.args:
        await update.message.reply_text("사용법: /news <검색어>")
        return

    keyword = ' '.join(context.args)
    url = f"https://newsapi.org/v2/everything?q={keyword}&language=ko&apiKey={ApiNewsKey}"

    try:
        response = requests.get(url)
        data = response.json()

        if 'articles' not in data:
            await update.message.reply_text("뉴스를 가져오는 데 실패했습니다.")
            return

        for i, article in enumerate(data['articles'][:5]):  # 상위 5개 뉴스만
            title = article['title']
            desc = article['description'] or "설명 없음"
            await update.message.reply_text(f"{i+1}. {title}\n{desc}\n")
    except Exception as e:
        await update.message.reply_text("오류 발생: " + str(e))

# --- 기존 코드 (start 함수만 수정) ---
async def start(update: Update, context: object) -> None:
    """/start 명령어를 받았을 때 응답합니다."""
    user = update.effective_user

    # 1. 이미지 보내기 예시 (이전과 동일)
    image_url = "https://cdn.imweb.me/upload/S20210103c5204f5ed12ce/8017dcc999129.png"
    await context.bot.send_photo(
        chat_id=update.effective_chat.id,
        photo=image_url,
        caption="저희 봇의 로고입니다!",
    )
    logger.info(f"Sent image to user {user.first_name}.")

    # --- 새로운 부분: API 호출하여 정보 가져오기 ---
    api_info_result = await get_api_data()  # 위에서 정의한 API 호출 함수 실행

    # 2. 소개 페이지 (HTML 형식의 꾸며진 텍스트와 링크)
    introduction_text = f"""
안녕하세요, <b>{user.mention_html()}</b>님! 👋
저는 미래채움 <b>봇</b>입니다.

{api_info_result} # API 호출 결과가 여기에 삽입됩니다!

이 봇은 다음과 같은 기능을 제공해요:
•  <code>/start</code>: 원주고 이미지와, 날씨 정보를 제공합니다
• 에코기능이 있어서 따라 따라말하는 기능이 있습니다.
•  <i>앞으로 더 많은 기능이 추가될 예정입니다 감사합니다</i>

    """

    await update.message.reply_html(introduction_text)
    logger.info(f"User {user.first_name} started the bot and received intro with API data.")


# 나머지 echo, error_handler, main 함수는 이전과 동일합니다.
async def echo(update: Update, context: object) -> None:
    """사용자의 메시지를 그대로 다시 보냅니다."""
    await update.message.reply_text(update.message.text)
    logger.info(f"Echoed message: {update.message.text}")


async def weather_command(update: Update, context: object) -> None:
    """/weather 명령어를 받았을 때 날씨 정보를 응답합니다."""
    user = update.effective_user
    logger.info(f"User {user.first_name} requested weather info.")

    # get_api_data() 함수를 호출하여 날씨 정보를 가져옵니다.
    weather_info = await get_api_data()

    # 가져온 날씨 정보를 사용자에게 메시지로 보냅니다.
    await update.message.reply_text(weather_info)


async def error_handler(update: Update, context: object) -> None:
    """봇의 오류를 로깅합니다."""
    logger.warning(f"업데이트 {update}에서 오류가 발생했습니다: {context.error}")


async def main() -> None:
    """봇을 실행합니다."""
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("weather", weather_command))  # <-- 이 줄 추가!
    application.add_handler(CommandHandler("news", news_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))
    application.add_error_handler(error_handler)

    logger.info("봇을 시작합니다...")
    application.run_polling()
    logger.info("봇이 종료되었습니다.")



if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if "cannot close a running event loop" in str(e).lower() or "cannot be called from a running event loop" in str(e).lower():
            nest_asyncio.apply()
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main())  
        else:
            raise

ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 669

RuntimeError: Cannot close a running event loop